In [35]:
import dash
from dash import html as html
import dash_core_components as dcc
import plotly.graph_objects as go
import plotly.express as px
import datetime
import time
import numpy as np
import pandas as pd
import requests

In [57]:
def get_price(tick,res,limit,start):
    #start is the startime of our trade in epoch time
    start=str(start)
    res=str(res)
    hist=requests.get('https://ftx.com/api/markets/'+tick+'-PERP/candles?resolution='+res+'&start_time='+start+'&limit='+limit).json()
    hist=pd.DataFrame(hist['result'])
    hist.drop(['startTime'], axis = 1, inplace=True)
    return hist

def get_price_24h(tick,res):
    t=time.time()-86400
    t=np.round(t)
    data=get_price(tick,res,'5000',t)
    return data

#get prices over the last delta seconds
def get_price_delta(tick,res,delta):
    t=time.time()-delta
    t=np.round(t)
    data=get_price(tick,res,'5000',t)
    return data

In [96]:
def graph_tick(tick,res,delta):
    # Function for creating line chart showing ticker prices over the last delta seconds
    df=get_price_delta(tick,res,delta)
    aux=df.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    fig = go.Figure([go.Scatter(x = aux['time'], y = df['close'],\
                     line = dict(color = 'firebrick', width = 4), name = tick)
                     ])
    fig.update_layout(title = 'Prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig
def graph(df):
    # Function for creating line chart showing spread prices
    aux=df.copy(deep=True)
    aux['time'] = pd.to_datetime(aux['time'], unit='ms')
    fig = go.Figure([go.Scatter(x = aux['time'], y = df['close'],\
                     line = dict(color = 'firebrick', width = 4), name = 'Spread')
                     ])
    fig.update_layout(title = 'Spread over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Spread'
                      )
    return fig
#Get spread between two assets given a cointegration coefficient, resolution since actual time-delta seconds
def get_spread(tick_1,tick_2,lam12,res,delta):
    df1=get_price_delta(tick_1,res,delta)
    df2=get_price_delta(tick_2,res,delta)
    dfaux=[df1['time'],np.log10(df1['close'])-lam12*np.log10(df2['close'])]
    headers = ['time','close']
    dfs = pd.concat(dfaux, axis=1, keys=headers)
    return dfs

In [106]:
app = dash.Dash()   #initialising dash app
tick1='AR'
tick2='SECO'
lam12=1.2
res=300
delta=5*86400
sp=get_spread(tick1,tick2,lam12,res,delta)

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Spread of '+tick1+' and '+tick2+' with lambda= '+str(lam12), style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),

        
        dcc.Graph(id = 'line_plot', figure = graph(sp))    
    ]
                     )

In [107]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Jan/2022 13:02:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 13:02:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 13:02:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 13:02:15] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2022 13:02:15] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


# Tests